# Swap LRNA
Agent $\alpha$ swaps asset $i$ with the protocol for LRNA.

## Constraints

The swap will keep the swap invariant for asset $i$, $R_iQ_i$, constant.

## Requirements

#### Case 1: Asset $i$ sold
$$
\Delta r^\alpha \leq r_i^\alpha
$$
#### Case 2: LRNA sold
$$
-\Delta q^\alpha \leq q^\alpha
$$

## Updating AMM state

### Case 1: LRNA sold, $\Delta q^\alpha < 0$ specified

If $-\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.

$$
\begin{align}
\Delta Q_i &= -\Delta q^\alpha\\
\Delta R_i &= R_i\frac{- \Delta Q_i}{Q_i + \Delta Q_i}(1 - f_A)\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i}{Q_i + \Delta Q_i}\right)\\
\Delta r_i^\alpha &= - \Delta R_i\\
\end{align}
$$

### Case 2: Asset $i$ bought, $\Delta r_i^\alpha > 0$ specified

$$
\begin{align}
\Delta R_i &= -\Delta r_i^\alpha\\
\Delta Q_i &= Q_i\frac{-\Delta R_i}{R_i(1 - f_A) + \Delta R_i}\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i}{Q_i + \Delta Q_i}\right)\\
\Delta q^\alpha &= - \Delta Q_i\\
\end{align}
$$

If $-\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.